In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import xarray as xr

In [2]:
path = "~/Downloads/sst1970-1979.nc"

ds = xr.open_dataset(path)

In [3]:
sst = ds['sst']
sst = sst.transpose('lat','lon','time')
sst

<xarray.DataArray 'sst' (lat: 180, lon: 360, time: 120)>
[7776000 values with dtype=float32]
Coordinates:
  * lon      (lon) float32 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat      (lat) float32 89.5 88.5 87.5 86.5 85.5 ... -86.5 -87.5 -88.5 -89.5
  * time     (time) datetime64[ns] 1970-01-01T12:00:00 ... 1979-12-01T12:00:00
Attributes:
    long_name:     Monthly 1 degree resolution SST
    units:         degrees C
    actual_range:  [-1000.         33.53844]
    description:   HadISST sea surface temperature. values of -1000 indicate ...

In [4]:
def clmMonLLT(x):
    
    # Calculate the sizes of time dimension
    len_of_dim = x.sizes
    time_size = len_of_dim[x.dims[2]]
    rank = len(len_of_dim)

    # Check if rank of dataarray matches the function
    if (rank != 3):
        print("Expected variable of rank = 3, recieved rank = {}".format(rank))
        return None
    
    # Check if number of months are multiple of 12; if not, exit the function
    no_of_months = 12
    if ((time_size % no_of_months) != 0):
        print("clmMonTLL: dimension must be a multiple of 12")
        return None
    
    # Get the sizes of latitude and longitude dimension
    lat_size = len_of_dim[x.dims[0]]
    lon_size = len_of_dim[x.dims[1]]
    
    # Store the time dimension name present in dataset as time variable
    time = x.dims[2]
    
    # Store as a string for groupby operation
    time_month = time + '.month'
    
    # Compute 12 months average
    ave_month = x.groupby(time_month).mean(time)
    
    # Copy and update the attributes
    ave_month.attrs = x.attrs
    ave_month = ave_month.rename("aveMonth")
    ave_month.attrs['time_op_ncl'] = "Climatology: " + str(int(time_size/no_of_months)) + " years"
    ave_month.attrs['info'] = "function clmMonLLT"
    
    # Return the results
    return ave_month

In [5]:
def calcMonAnomLLT(x, xAve):
    # x is the dataarray and xAve is the monthly averages array of x
    
    len_of_dim = x.sizes
    no_of_time = len_of_dim[x.dims[2]] # no_of_time = Size of time dimension
    rank = len(len_of_dim)

    # Check if rank of dataarray matches the function
    if (rank != 3):
        print("Expected variable of rank = 3, recieved rank = {}".format(rank))
        return None
    
    
    # Check if number of months are multiple of 12; if not, exit the function
    no_of_months = 12
    if ((no_of_time % no_of_months) != 0):
        print("calcMonAnom: dimension must be a multiple of 12")
        return None
    
    # Store the time dimension name present in dataset as time variable
    time = x.dims[2]
    
    # Store as a string for groupby operation
    time_month = time + '.month'
    
    
    # Calculate anomalies by subtracting monthly means from actual dataarray
    xAnom = x.groupby(time_month) - xAve
        
    # Copy and update the attributes
    xAnom.attrs = x.attrs
    xAnom.attrs['anomaly_op_ncl'] = "Anomalies from Annual Cycle: calcMonAnomLLT"
    
    # Drop the extra month co-ordinate from xAnom DataArray
    xAnom = xAnom.drop('month')
    xAnom = xAnom.rename('xAnom')
    
    # Return the new dataarray
    return xAnom

In [6]:
xAve = clmMonLLT(sst)

In [7]:
xAve

<xarray.DataArray 'aveMonth' (lat: 180, lon: 360, month: 12)>
array([[[-900.18, -900.18, ..., -900.18, -900.18],
        [-900.18, -900.18, ..., -900.18, -900.18],
        ...,
        [-900.18, -900.18, ..., -900.18, -900.18],
        [-900.18, -900.18, ..., -900.18, -900.18]],

       [[-900.18, -900.18, ..., -800.36, -800.36],
        [-900.18, -900.18, ..., -800.36, -800.36],
        ...,
        [-900.18, -900.18, ..., -800.36, -800.36],
        [-900.18, -900.18, ..., -800.36, -800.36]],

       ...,

       [[    nan,     nan, ...,     nan,     nan],
        [    nan,     nan, ...,     nan,     nan],
        ...,
        [    nan,     nan, ...,     nan,     nan],
        [    nan,     nan, ...,     nan,     nan]],

       [[    nan,     nan, ...,     nan,     nan],
        [    nan,     nan, ...,     nan,     nan],
        ...,
        [    nan,     nan, ...,     nan,     nan],
        [    nan,     nan, ...,     nan,     nan]]], dtype=float32)
Coordinates:
  * lon      (lon) fl

In [8]:
result = calcMonAnomLLT(sst, xAve)

In [9]:
result

<xarray.DataArray 'xAnom' (lat: 180, lon: 360, time: 120)>
array([[[ -99.82001,  -99.82001, ...,  -99.82001,  -99.82001],
        [ -99.82001,  -99.82001, ...,  -99.82001,  -99.82001],
        ...,
        [ -99.82001,  -99.82001, ...,  -99.82001,  -99.82001],
        [ -99.82001,  -99.82001, ...,  -99.82001,  -99.82001]],

       [[ -99.82001,  -99.82001, ..., -199.64001, -199.64001],
        [ -99.82001,  -99.82001, ..., -199.64001, -199.64001],
        ...,
        [ -99.82001,  -99.82001, ..., -199.64001, -199.64001],
        [ -99.82001,  -99.82001, ..., -199.64001, -199.64001]],

       ...,

       [[       nan,        nan, ...,        nan,        nan],
        [       nan,        nan, ...,        nan,        nan],
        ...,
        [       nan,        nan, ...,        nan,        nan],
        [       nan,        nan, ...,        nan,        nan]],

       [[       nan,        nan, ...,        nan,        nan],
        [       nan,        nan, ...,        nan,        nan],
 

In [10]:
result[99,99,0:120:12]

<xarray.DataArray 'xAnom' (time: 10)>
array([ 0.563919, -2.808434, -0.202984,  1.435978, -0.6576  , -0.732443,
       -0.595802,  1.451853,  0.552422,  0.993099], dtype=float32)
Coordinates:
    lon      float32 -80.5
    lat      float32 -9.5
  * time     (time) datetime64[ns] 1970-01-01T12:00:00 ... 1979-01-01T12:00:00
Attributes:
    long_name:       Monthly 1 degree resolution SST
    units:           degrees C
    actual_range:    [-1000.         33.53844]
    description:     HadISST sea surface temperature. values of -1000 indicat...
    anomaly_op_ncl:  Anomalies from Annual Cycle: calcMonAnomLLT